In [4]:
search_for = 123
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.003944873809814453
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 350465005
type: [1, 1, 1, 1, 123] 123
cases of this type: 228886641
100000 13.516355059516817
200000 23.595864647450963
300000 25.642013411360118
400000 31.46961366025325
500000 34.00007365869081
600000 29.24220272936314
700000 24.654045929754762
800000 31.163162760188474
900000 29.413322569529658
1000000 13.608955771177472
1100000 25.425801662603583
1200000 27.213389363068565
1300000 27.28298894598465
1400000 25.24582150733396
1500000 26.897469665834127
1600000 24.440073396964074
1700000 26.857968642103568
1800000 25.114897462417915
1900000 10.013849589804858
2000000 27.925577224137317
2100000 30.55947435641835
2200000 31.396247959470177
2300000 29.648532504674765
2400000 32.539122914141345
2500000 28.637728026428952
2600000 30.147651954390838
2700000 28.782603337159554
2800000 19.56640884782931
2900000 22.57300024334048
300000

29300000 23.270315942379575
29400000 23.845686971435395
29500000 25.588575126055296
29600000 12.079323159445462
29700000 25.348605455522616
29800000 24.276018456906318
29900000 25.58490961461918
30000000 22.759041733542436
30100000 12.989021532466223
30200000 26.05433086033672
30300000 23.505345520036002
30400000 24.8025341746518
30500000 22.01783351153975
30600000 20.582153582866898
30700000 17.788626687460333
30800000 14.79672579941776
30900000 20.74180910247021
31000000 23.540411606293965
31100000 22.616880721132038
31200000 24.776788373257396
31300000 23.795748454054795
31400000 15.50171525480886
31500000 21.75699583761729
31600000 23.4230353988944
31700000 25.981997469608746
31800000 23.54161828136993
31900000 19.488092659657468
32000000 18.179556085236786
32100000 24.54809970840923
32200000 24.769104029100575
32300000 23.740949429478725
32400000 23.452792358191143
32500000 20.898117300079257
32600000 8.786600369125134
32700000 23.92454357030382
32800000 20.9619574550621
32900000 

59000000 18.521717781079182
59100000 10.983658947109973
59200000 21.54787195635011
59300000 19.926470857903258
59400000 22.641892391323
59500000 20.95286305128175
59600000 23.556645480828653
59700000 21.04469595178497
59800000 21.9882917025354
59900000 20.788918442294484
60000000 20.689900097360557
60100000 10.676227614350696
60200000 18.913400997853667
60300000 18.710535193963715
60400000 20.37976695482269
60500000 9.59930181992362
60600000 16.94381784601291
60700000 20.689572154039205
60800000 18.695391900755588
60900000 17.756454100614818
61000000 13.818735881283214
61100000 19.89837213697914
61200000 22.178838254794385
61300000 20.308329632955015
61400000 22.302468731768805
61500000 21.54850743060295
61600000 21.400519653139423
61700000 22.542149230568548
61800000 20.06797794335291
61900000 18.53580754253181
62000000 12.043176851680371
62100000 18.800105894174628
62200000 21.32897283634028
62300000 19.481180674973487
62400000 7.625830691990895
62500000 20.834666320803862
62600000 1

88800000 17.98220911972102
88900000 18.309216921404122
89000000 17.29494809004201
89100000 19.02141906052141
89200000 18.8579662499505
89300000 20.452810052976997
89400000 18.978342617001992
89500000 19.53873447684633
89600000 18.425766043165556
89700000 16.730356827820238
89800000 18.32126603647823
89900000 17.484061894216396
90000000 17.571354217053855
90100000 8.638275723137587
90200000 16.47256463550189
90300000 6.401995397137251
90400000 18.668171564271937
90500000 8.730612562222081
90600000 16.88834561454912
90700000 18.637721007275722
90800000 17.032129868377137
90900000 18.042236534313517
91000000 19.045103068048597
91100000 18.648422232266338
91200000 19.98269863025503
91300000 20.04535049306816
91400000 19.643225295367202
91500000 16.820109430932845
91600000 18.53248525941458
91700000 17.262282179836195
91800000 17.055397735583743
91900000 17.90967357888365
92000000 8.555817058096263
92100000 13.679488853502074
92200000 11.917695286277095
92300000 11.660105295910938
92400000 

117900000 1.77167099236692
118000000 1.7718808094231489
118100000 1.795129475309031
118200000 1.7730416701419585
118300000 1.755250963663926
118400000 1.7574402539595135
118500000 1.7762004619107379
118600000 1.7706101000294188
118700000 1.7659468465253612
118800000 1.752200936544339
118900000 1.8027710771169962
119000000 1.7527278004713356
119100000 2.601811114318205
119200000 23.142926532113524
119300000 21.95397348622951
119400000 21.62906255155561
119500000 21.023469692118844
119600000 21.248414861792092
119700000 20.803036358380105
119800000 20.460889336120513
119900000 21.005966405137315
120000000 10.680460805365527
120100000 2.7064185463934445
120200000 19.129308183929428
120300000 19.35984387663408
120400000 21.0927749077641
120500000 20.24931418386078
120600000 21.709530999904604
120700000 21.1169555284068
120800000 21.75446612497613
120900000 21.898034319182067
121000000 21.5652812884652
121100000 17.971560269312345
121200000 17.513164466170714
121300000 16.643322168123344
12

...

233000000 2.678042540460149
233100000 2.2318205095704924
233200000 2.5212998131984774
233300000 2.1905588558831313
233400000 2.1760642281396954
233500000 1.1270095796453654
233600000 0.524239138442311
233700000 1.5202391705115934
233800000 2.1224296342822413
233900000 2.3237103987490935
234000000 2.555986552936968
234100000 2.354959847300917
234200000 2.079111580605987
234300000 2.101210422692024
234400000 2.001461139470468
234500000 2.319540808403237
234600000 2.349751153230303
234700000 1.9074392162335119
234800000 2.986479809708122
234900000 2.1153601296308175
235000000 2.16276231773594
235100000 2.117934954287251
235200000 2.016759627763311
235300000 3.3080141168368464
235400000 0.5956018952258163
235500000 0.5152524130884839
235600000 2.0955221758370004
235700000 1.9871711446641982
235800000 2.192705314581
235900000 2.678635970834285
236000000 2.07257088222449
236100000 1.9619661499316097
236200000 1.8649382309951288
236300000 3.655339956186298
236400000 4.039268554294477
23650000

261800000 3.2084262081680492
261900000 3.6251276824672782
262000000 3.0220082850840986
262100000 3.06286247877973
262200000 2.9995937930777705
262300000 3.2472957545369763
262400000 4.152619759861625
262500000 3.780550254101356
262600000 3.412871311163833
262700000 3.224468705373473
262800000 3.5040416547023883
262900000 3.2860240116595927
263000000 3.1948059585638675
263100000 3.005998891228997
263200000 3.911465347628534
263300000 0.7286876470909285
263400000 0.7257499794116417
263500000 2.7570725700136425
263600000 3.2504309561031
263700000 3.2001379235393177
263800000 3.90802937452966
263900000 3.186895839756989
264000000 2.929681917996572
264100000 3.3024303984560572
264200000 3.6115584587134553
264300000 4.527599516611407
264400000 4.006251973943074
264500000 3.5525821359556122
264600000 3.8894192393920193
264700000 3.6244532890713512
264800000 3.7468749573072064
264900000 3.4725232381256426
265000000 3.6302242642569444
265100000 3.2085091407550936
265200000 0.7054569769033691
26

290600000 2.088314489340332
290700000 1.9107711766832571
290800000 2.2473335587102174
290900000 2.355300581008007
291000000 1.9540702135503687
291100000 2.0651351401863196
291200000 0.7834104168039989
291300000 0.4937954408796231
291400000 1.8126573073073526
291500000 2.1939530062005455
291600000 1.986245687661224
291700000 2.1286059134334328
291800000 2.132552651069939
291900000 2.013685147797866
292000000 2.347740537746221
292100000 2.139419054831018
292200000 2.5083497386289735
292300000 2.167522729480813
292400000 2.435317277756416
292500000 1.8471330473541383
292600000 2.0354163221977055
292700000 2.068273911438667
292800000 2.3103953488878175
292900000 2.100249664337287
293000000 1.6609765874239837
293100000 0.4783470447459121
293200000 0.4775978659231862
293300000 2.370098179388321
293400000 2.0582724366791645
293500000 2.255388650485933
293600000 2.0203306018422014
293700000 2.104915399100254
293800000 1.8993108349768224
293900000 2.43921016552188
294000000 2.139620688780503
29

318400000 1.2640321226570408
318500000 1.1722291129107774
318600000 1.2955811045804786
318700000 1.2385172574649752
318800000 1.2105570817674367
318900000 1.1393513766071002
319000000 1.1877453335284491
319100000 1.1355424521876474
319200000 1.3403326798035868
319300000 1.1696626894650957
319400000 1.1810681862703463
319500000 1.2240157561891443
319600000 1.1992464316468536
319700000 1.138668931629863
319800000 1.3406560349666543
319900000 1.1448510613907477
320000000 1.1947745860420964
320100000 1.1469428614497383
320200000 1.171684562419653
320300000 1.1384846667460733
320400000 1.244571245285809
320500000 1.2118104945078492
320600000 1.1284091501547695
320700000 1.1092110098109775
320800000 1.1865842157243491
320900000 1.076228689290921
321000000 1.1037119607901937
321100000 1.1837911203357079
321200000 1.209034372972174
321300000 1.1826220857980283
321400000 1.060863124220431
321500000 0.9651907870356241
321600000 1.1331312082274723
321700000 1.1061239630921185
321800000 1.07439728

345300000 0.1267488284443617
345400000 0.11942824315774442
345500000 0.159749985399882
345600000 0.08562478025861912
345700000 0.10570850124701858
345800000 0.10662036548534036
345900000 0.13476045782435936
346000000 0.1441491803312699
346100000 0.1298953641242782
346200000 0.15979107092551723
346300000 0.09775933611370126
346400000 0.1443928513058126
346500000 0.11963237112192973
346600000 0.14081767946326731
346700000 0.1318505489344696
346800000 0.10410873448806338
346900000 0.11324023067498207
347000000 0.0838844116211335
347100000 0.1071405398025877
347200000 0.07940489266632994
347300000 0.08523357538691495
best so far: 0
type: [1, 3, 1, 41, 1] 123
cases of this type: 206763
347400000 0.05568387730652426
347500000 0.01393480997416377
best so far: 0
type: [1, 3, 41, 1, 1] 123
cases of this type: 5043
best so far: 0
type: [1, 41, 1, 1, 3] 123
cases of this type: 3321
best so far: 0
type: [1, 41, 1, 3, 1] 123
cases of this type: 1107
best so far: 0
type: [1, 41, 3, 1, 1] 123
cases o